In [0]:
import tensorflow as tf
import numpy as np
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc

import six

from tensorflow.contrib.seq2seq.python.ops import decoder
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import embedding_ops
from tensorflow.python.ops import gen_array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import tensor_array_ops
from tensorflow.python.ops.distributions import bernoulli
from tensorflow.python.ops.distributions import categorical
from tensorflow.python.util import nest
_transpose_batch_time = decoder._transpose_batch_time

In [0]:
tf.reset_default_graph()
input_data = np.random.randint(0,10,[2,5])
sentence_length = [len(i) for i in input_data]
input_place = tf.placeholder(name='inpu',dtype=tf.int32,shape=[2,5])  #batch_major input
sequence_length_s= tf.placeholder(name='seq_len',dtype=tf.int32,shape=[None])
custom_embedding = tf.get_variable(name='embedding',shape=[10,50],dtype=tf.float32,initializer=tf.random_uniform_initializer(-0.01,0.01))



In [0]:
def _unstack_ta(inp):
  return tensor_array_ops.TensorArray(
      dtype=inp.dtype, size=array_ops.shape(inp)[0],
      element_shape=inp.get_shape()[1:]).unstack(inp)



class TrainingHelper():
  """A helper for use during training.  Only reads inputs.
  Returned sample_ids are the argmax of the RNN output logits.
  """

  def __init__(self, inputs, sequence_length, time_major=False, name=None):
    """Initializer.
    Args:
      inputs: A (structure of) input tensors.
      sequence_length: An int32 vector tensor.
      time_major: Python bool.  Whether the tensors in `inputs` are time major.
        If `False` (default), they are assumed to be batch major.
      name: Name scope for any created operations.
    Raises:
      ValueError: if `sequence_length` is not a 1D tensor.
    """
    with ops.name_scope(name, "TrainingHelper", [inputs, sequence_length]):
      
      
      
      inputs = ops.convert_to_tensor(inputs, name="inputs")
      self._inputs = inputs
      if not time_major:
        inputs = nest.map_structure(_transpose_batch_time, inputs)
        
         

      self._input_tas = nest.map_structure(_unstack_ta, inputs)
       

      self._sequence_length = ops.convert_to_tensor(
          sequence_length, name="sequence_length")
      

      if self._sequence_length.get_shape().ndims != 1:
        raise ValueError(
            "Expected sequence_length to be a vector, but received shape: %s" %
            self._sequence_length.get_shape())

      self._zero_inputs = nest.map_structure(
          lambda inp: array_ops.zeros_like(inp[0, :]), inputs)

      self._batch_size = array_ops.size(sequence_length)

  @property
  def inputs(self):
    return self._inputs

  @property
  def sequence_length(self):
    return self._sequence_length

  @property
  def batch_size(self):
    return self._batch_size

  @property
  def sample_ids_shape(self):
    return tensor_shape.TensorShape([])

  @property
  def sample_ids_dtype(self):
    return dtypes.int32

  def initialize(self, name=None):
    with ops.name_scope(name, "TrainingHelperInitialize"):
      finished = math_ops.equal(0, self._sequence_length)
      print("dd",finished)
      all_finished = math_ops.reduce_all(finished)
      next_inputs = control_flow_ops.cond(
          all_finished, lambda: self._zero_inputs,
          lambda: nest.map_structure(lambda inp: inp.read(0), self._input_tas))
      return (finished, next_inputs)

In [17]:
embedd_input = tf.nn.embedding_lookup(custom_embedding,input_place)

helper_fun = TrainingHelper(inputs=input_place,sequence_length=sequence_length_s,time_major=False)

operation = tf.add(input_place,input_place)

ok=helper_fun.initialize(name=None)



dd Tensor("TrainingHelperInitialize_4/Equal:0", shape=(?,), dtype=bool)


In [20]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  result,b=sess.run([operation,ok],feed_dict={input_place:input_data,sequence_length_s:sentence_length})
  print(b,embedd_input.shape)


(array([False, False]), array([7, 7], dtype=int32)) (2, 5, 50)
